# Bristle Simulation

In [ ]:
# Import Modules
import numpy as np
import numpy.linalg as la

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

from tqdm.notebook import tqdm

from cosserat import Cosserat
from bspline import test_bspline, snake_bspline


In [ ]:
%load_ext autoreload
%autoreload 2

### Contact Force (Wall Force)

In [ ]:

# Function needs n = num_of_elements, f_element, current_r, and current_v and z_wall (NEW PARAMETER)
def wall_force (n, f_gravity, r, v):
    # Extract the y-components of gravity for each element
    #set f_gravity to zero before passing into wall_force
    f_element = f_gravity [:, 1] # should ideally be a 3-dim vector of zeros
    
    # wall stiffness and dissipation constants
    k_w = 10000000
    gamma_w = 1e-6
    
    # Normal vector by the wall (points in negative z - direction)
    wall_normal = np.zeros((n, 3, )) # (n,3, )
    wall_normal[:] = [0., 0., -1.]

    # Penetration distance: epsilon
    z_wall = 1+np.sqrt(0.5); # wall at position 0
    z_coord = r[:, 2]   # A vector of z - coordinates of each node: dimensions: (n+1, )
    z_coord_elem = (z_coord [:-1] + z_coord [1:]) / 2.  # A vector of z - coordinates of each node: dimensions: (n+1, )
    epsilon = z_coord_elem - np.ones((n, )) * z_wall  # Penetration depth of each element's centre: (n,)

    #Velocity of the centre of each element (in z - direction only)
    velo_element = (v [1:,2] + v [:-1,2])/2. # (n, )

    # wall_force on each element: ((n, 3, ))
    f_wall_elem = np.zeros((n, 3, ))
    # updating just the z-component of wall force as no force is exerted along x and z axis
    f_wall_elem [:, 2] = np.heaviside(epsilon, 1.) * ( - f_element - k_w * epsilon - gamma_w * velo_element)
    
    # wall_force on each node: ((n+1, 3, ))
    f_wall_nodes = np.zeros((n+1, 3,))
    f_wall_nodes[:-1] += (f_wall_elem)/2.
    f_wall_nodes[1:] += (f_wall_elem)/2.

    return f_wall_nodes, f_wall_elem

### Anisotropic Friction

In [ ]:
# Longitudinal Friction Calculation
# calcs friction for each node
def friction_long (v, t, f_element, f_wall_element, n, mu_sf, mu_kf, mu_sb, mu_kb):
    wall_normal = np.zeros((n, 3, ))
    wall_normal[:,2] = -1.0
    
    u_para = np.cross(t, wall_normal) # (n,3)
    u_x = np.cross(wall_normal, u_para) 
    u_x = u_x / la.norm (u_x, axis = 1, keepdims = True) # (n,3) 
    
    # get velocity from the cosserat class: v will be of shape: (n+1,3)
    # convert v to (n, 3, ) -> Avg velocity of the adjacent nodes is assigned to the centre of the element    
    v_elem = (v [:-1] + v [1:])/2. # (n,3)
    v_epsilon = 1e-6  # (threshold velocity)    
    
    vx_mag = np.sum(v_elem * u_x, axis=1, keepdims=True)
    
    mask = np.abs(vx_mag)[:,0] > v_epsilon # masking static friction (True element applies kinetic friction)
    f_long_elem = u_x.copy()
    _abs_f_wall_element = np.abs(f_wall_element[:,1:2])
    # Kinetic friction
    if np.any(mask):
        direction = -np.sign(vx_mag[mask])
        mu = np.ones_like(direction) * mu_kb
        mu[direction < 0] = mu_kf
        f_long_elem[mask] *= (direction * mu * _abs_f_wall_element[mask])

    # Static friction
    if np.any(~mask):
        f_dot_ux = np.sum(f_element[~mask]*u_x[~mask], axis=1, keepdims=True)
        direction = -np.sign(f_dot_ux)
        mu = np.ones_like(direction) * mu_sb
        mu[direction < 0] = mu_sf
        f_long_elem[~mask] *= (direction * np.fmin(np.abs(f_dot_ux), mu * _abs_f_wall_element[~mask]))
    
    f_long_node = np.zeros((n+1, 3, ))
    f_long_node[:-1] += (f_long_elem)/2.
    f_long_node[1:] += (f_long_elem)/2.
    
    
    return f_long_node

In [ ]:
# Generate Animation
def make_animation(s_list, r_list, n_step):
    fig = plt.figure() 
    #ax = plt.axes(xlim=(-50, 50), ylim=(-50, 50)) 
    ax1 = fig.add_subplot(111)
    plot, = ax1.plot([],[],'-x')
    xmax = max([np.max(ar) for ar in s_list])
    xmin = min([np.min(ar) for ar in s_list])
    ymax = max([np.max(ar) for ar in r_list])
    ymin = min([np.min(ar) for ar in r_list])
    plt.title('xz plot')
    plt.xlabel('x')
    plt.ylabel('z')
    plt.xlim([xmin,xmax])
    plt.ylim([ymin,ymax])
    def animate(i):
        _ = plot.set_data(s_list[i], r_list[i])

    anim = animation.FuncAnimation(fig, animate, frames=n_step) 
    return fig, anim

# CMAES Optimization Block:

In [ ]:
from project3_setup import setup, L, n, M, mu_sf, mu_kf, mu_sb, mu_kb, dt, T_m

#s_hat = np.linspace(0., L, n+1) # Positions of nodes along centerline when t = 0 (i.e. initial configuration)
#s_elements = (s_hat[:-1] + s_hat[1:])/2. # [n] --> Center points of each element along centerline

s_list = [] # Position list for future use
y_list = [] # Height list for future use
t_list = [] # time list
friction = []
total_force_list = []

def bristle_CMA_call():
    
    # initialization
    solution = Cosserat(**setup)
    n_step = 100000
    r, v, Q, w = solution.get_state # Initial state
    
    print(r)
    
    tangent = solution.get_tangent
    
    s_list.append(r[:,2]) # save k-axis
    y_list.append(r[:,0]) # save i-axis
    
    # Get the parameter x from CMAES

    
    #Initialize internal shear force at each element
    shear_nodes = np.zeros((n+1,3,))
    
    time = 0
    for itr in tqdm(range(n_step)):
        
        shear_nodes[0], shear_nodes[-1]  = 2 * shear_nodes[0], 2 * shear_nodes[-1]  
        # Calc shear at the elements: [n]
        shear_elements = (shear_nodes [:-1] + shear_nodes[1:])/2.
               
        # (i) Wall Force Calculation:
        f_wall_nodes, f_wall_elements = wall_force (n, shear_elements, r, v)
        
        # (ii) Friction Force Calculation:
        F = f_wall_elements + shear_elements # Total force on each element
        #f_friction_nodes = friction_long (v, tangent, F, f_wall_elements, n, mu_sf, mu_kf, mu_sb, mu_kb)
        #friction.append(np.average(np.linalg.norm(f_friction_nodes[:], axis = 1)))
            
        # (iii) Total Force Calculation at each node:
        f_total_nodes = f_wall_nodes #+ f_friction_nodes
        #print("w/o damping", np.linalg.norm(f_total_nodes[:], axis = 1))
        
        #print(f_total_nodes)
        
        f_total_nodes -= 1000 * v
        #print("with damping", np.linalg.norm(f_total_nodes[:], axis = 1))
        
        # Run step
        r, v, Q, w  = solution.step(dt = dt, force = f_total_nodes)

        tangent = solution.get_tangent
        shear_nodes = solution.get_shear
        total_force_list.append(f_total_nodes[-1]+shear_nodes[-1])
         
        #print(r)
        
        if itr % 250 == 0:
            s_list.append(r[:,2])
            y_list.append(r[:,0])
            t_list.append(time)

        time += dt
        
#     plt.plot(np.array(total_force_list), label = 'Total Force')
#     #plt.plot(np.array(friction)[:,1], label = 'Friction Z')
#     #plt.title("Average friction on snake at any iteration")
#     #plt.title("Friction on last element of snake at any iteration")
#     plt.legend()
#     plt.ylabel("Force (N)")
#     plt.xlabel("Iterations")
#     plt.plot()
    
    return r

In [ ]:
# Test optimal run

bristle_CMA_call()

In [ ]:
# plt.plot(np.array(total_force_list).T[0])
# #plt.plot(np.array(friction)[:,1], label = 'Friction Z')
# #plt.title("Average friction on snake at any iteration")
# #plt.title("Friction on last element of snake at any iteration")
# #plt.legend()
# plt.ylabel("Force (N)")
# plt.xlabel("Iterations")
# plt.plot()

In [ ]:
_, anim = make_animation(y_list, s_list, len(s_list))

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
print(total_force_list[-100])